In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/lincong/miniconda3/envs/gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
path = '/cognitive_comp/lincong/models/autoj-13b'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(path, padding_side='right')

In [5]:
tokenizer.pad_token_id

32000

In [6]:
model = AutoModelForCausalLM.from_pretrained(path).cuda()

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:38<00:00, 12.69s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /cognitive_comp/lincong/models/autoj-13b and are newly initialized: ['model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.39.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.32.self_attn.rotary_emb.inv_freq', 'model.layers.34.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.37.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'mode

In [7]:
reflector_template = {}
reflector_template['auto-j_13B'] = """Write critiques for a submitted response on a given user's query, and grade the response:
  
[BEGIN DATA]
***
[Query]: {__query__}
***
[Response]: {__response__}
***
[END DATA]

Write critiques for this response. After that, you should give a final rating for the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]"."""

In [24]:
    reflector_gen_args = {
        "do_sample": True,
        "top_p": 0.85,
        "top_k": 0,
        # "bad_words_ids": [],
        "max_new_tokens": 2048,
        "repetition_penalty": 1.0,
        "temperature": 0.85,
        "use_cache": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }

In [31]:
q = 'Do we spend only a small part of the US Federal budget on foreign aid to poor countries?'
r = """No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.

According to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.

To put this amount into perspective, $40.5 billion is more than the annual GDP of many countries around the world. It is also more than the total amount of money raised by all Kickstarter campaigns in 2020.

While $40.5 billion may seem like a lot of money, it is important to remember that it is a small fraction of the total federal budget. In fiscal year 2020, the federal government spent over $4 trillion, so foreign aid accounts for only about 1% of the total federal spending.

However, it is worth noting that foreign aid is a critical investment in the global community and can have a significant impact on poverty reduction, economic development, and humanitarian assistance. The money spent on foreign aid can help improve healthcare, education, and infrastructure in developing countries, and can also support humanitarian responses to natural disasters and conflicts.

In conclusion, while foreign aid is not a large percentage of the overall federal budget, it is still a significant amount of money that can have a meaningful impact on the global community. It is important to continue investing in foreign aid to support the world's most vulnerable populations and promote economic development and stability."""

In [32]:
input_ids = tokenizer(reflector_template['auto-j_13B'].format(__query__=q, __response__=r), return_tensors='pt').input_ids

In [33]:
tokenizer.batch_decode(input_ids)

['<s> Write critiques for a submitted response on a given user\'s query, and grade the response:\n  \n[BEGIN DATA]\n***\n[Query]: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?\n***\n[Response]: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.\n\nAccording to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.\n\nTo put this amount into perspective, $40.5 billion is more than the annual GDP of many countries around the world. It is also more t

In [38]:
output_ids = model.generate(input_ids.cuda(), **reflector_gen_args)

In [39]:
tokenizer.batch_decode(output_ids)

['<s> Write critiques for a submitted response on a given user\'s query, and grade the response:\n  \n[BEGIN DATA]\n***\n[Query]: Do we spend only a small part of the US Federal budget on foreign aid to poor countries?\n***\n[Response]: No, the amount of the US Federal budget spent on foreign aid to poor countries is not a small part. While it is true that foreign aid is a relatively small percentage of the overall federal budget, it is still a significant amount of money.\n\nAccording to the Office of Management and Budget (OMB), in fiscal year 2020, the US government spent approximately $40.5 billion on foreign aid, which is about 1.4% of the total federal budget. This includes funding for the State Department, the U.S. Agency for International Development (USAID), and other programs that support international development and humanitarian assistance.\n\nTo put this amount into perspective, $40.5 billion is more than the annual GDP of many countries around the world. It is also more t